In [1]:
import torch
import torch.nn as nn
from kasim_model import Kasim_Model
from kasim_tokenizer import Tokenizer   

k_tokenizer=Tokenizer("tokenizer.json")
sentence = "the capital of united states and the capital of france"
tokens = k_tokenizer.encode(sentence)
tokens = torch.tensor(tokens).unsqueeze(0)  # Add batch dimension


k_model = Kasim_Model(vocab_size=len(k_tokenizer.vocab), embed_dim=4, context_length=32)
k_model


C:\Users\kasim\AppData\Local\Temp\ipykernel_13756\1443349770.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  tokens = torch.tensor(tokens).unsqueeze(0)  # Add batch dimension


Kasim_Model(
  (embedding): Embedding(64, 4)
  (pos_embed): Embedding(32, 4)
  (self_attention): KasimCausalAttention(
    (q_weight): Linear(in_features=4, out_features=4, bias=False)
    (k_weight): Linear(in_features=4, out_features=4, bias=False)
    (v_weight): Linear(in_features=4, out_features=4, bias=False)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [2]:
sentence_meanings_whith_attention = k_model(tokens)
sentence_meanings_whith_attention

tensor([[[ 0.4248,  0.1705, -1.9001,  2.5001],
         [ 0.2162,  0.4196,  0.7077, -0.6772],
         [-0.2929,  0.0077,  0.3337, -0.5017],
         [ 0.1806,  0.0153,  0.0417, -0.1002],
         [-0.4780, -0.3526,  0.6592, -1.0815],
         [-0.1822, -0.1118, -0.3364,  0.3189],
         [-0.3784, -0.1647,  0.7853, -1.0333],
         [-0.1169, -0.0450,  0.9865, -1.1515],
         [-0.0236, -0.0915,  0.2125, -0.3132],
         [-0.0805, -0.0822,  0.3723, -0.4791],
         [-0.0320, -0.1930,  0.4371, -0.6443],
         [-0.2919, -0.6143,  0.6436, -1.0959],
         [-0.1000, -0.1537,  0.2449, -0.3743],
         [ 0.0220, -0.0403,  0.1353, -0.2665],
         [ 0.0203, -0.2313,  0.1045, -0.2241],
         [ 0.1158, -0.1941, -0.1055,  0.0604],
         [ 0.0639, -0.0784,  0.5117, -0.6560],
         [ 0.1312, -0.1224,  0.2375, -0.4013],
         [ 0.1090, -0.0534,  0.1966, -0.2077],
         [-0.0460, -0.2551,  0.3569, -0.5555]]], grad_fn=<UnsafeViewBackward0>)

Cosine benzerlik: 0.5855399966239929
